In [2]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
imagelist=['amber1.png','amber2.png','amy1.png','amy2.png','andrew1.png','andrew2.png','andy1.png','andy2.png','erin1.png','erin2.png','gabe1.png','gabe2.png','hill2.png','hill4.png','jack1.png','jack2.png','zach1.png','zach2.png']

train_img=[]
for file in imagelist:
    first=np.array(imageio.imread('Train/'+file))
    m,n=first.shape
    first=np.reshape(first,(m*n,1))
    train_img.append(first)

print(m,n)
img_train=np.matrix(np.array(train_img))
train_img=np.transpose(img_train)
print(train_img)
m,n=train_img.shape
print("train_img.shape=",m,n)

##calculating average face vector
avgface=np.ones((m,1))
for i in range(0,m):
    avgface[i]=np.mean(train_img[i])
print("The average face vector is",avgface)
storedavgface=avgface

##subtracting average face vector from all faces
for i in range(0,n):
    img_train[i]=img_train[i]-np.transpose(avgface)
print("img_train.shape=",img_train.shape)

150 125
[[ 28  22  13 ... 220 163 197]
 [ 38  20  20 ... 217 160 188]
 [ 38  26  27 ... 224 149 193]
 ...
 [ 14  15 174 ... 197 106  69]
 [  9  22 126 ... 237 123  72]
 [ 20  20 128 ... 252 123  75]]
train_img.shape= 18750 18
The average face vector is [[124.16666667]
 [129.11111111]
 [135.44444444]
 ...
 [ 81.66666667]
 [ 85.44444444]
 [ 90.05555556]]
img_train.shape= (18, 18750)


In [3]:
##calculating covariance matrix
covmat=(img_train*np.transpose(img_train))
print("The covariance matrix is")
#print(covmat)

##calculating eigen values out of covariance matrix
eigen,u=np.linalg.eig(covmat)
print("U matrix is")
#print(v)
print("eigen values are")
print(eigen)

a,s,v=np.linalg.svd(covmat)

##determining value of k i.e.(no. of principal components)
summation=np.sum(s)
print(summation)
for k in range(1,len(s)):
    sigma=0
    for i in range(0,k):
        sigma+=s[i]
    sigma=sigma/summation
    if(sigma>=0.95):
        print("selected value of k is",k)
        break

##selecting k eigen vectors from the set of 18 vectors
eigenvectors=np.empty((k,18))
for i in range(0,k):
    temp=np.empty((1,18))
    for j in range(0,18):
        temp[0,j]=u[i,j]
    eigenvectors[i]=temp

print("selected eigen vectors are",eigenvectors)

##mapping selected eigen vectors to original image size in training set
mappedeigenvectors=train_img*np.transpose(eigenvectors)
print(mappedeigenvectors.shape)

#temp=np.transpose(mappedeigenvectors)*mappedeigenvectors
#tempinv=np.linalg.inv(temp)
#weightmatrix=tempinv*np.transpose(mappedeigenvectors)*np.transpose(img_train)
#print(weightmatrix.shape)
weightmatrix=np.transpose(mappedeigenvectors)*np.transpose(img_train)
print(weightmatrix.shape)

The covariance matrix is
U matrix is
eigen values are
[2288.99062541  558.3637944   410.25292984  406.37000924 -459.74938166
 -420.98354155  268.4100919   255.17716935  185.34750369  105.48799671
   67.55609858 -326.56394201  -24.44091727 -291.51611385  -90.56536129
 -140.01107034 -178.65777665 -212.46811451]
6690.9124382579
selected value of k is 14
selected eigen vectors are [[-0.27328029 -0.19986906  0.05664229  0.01214961  0.11654215 -0.13742942
   0.06776537  0.15358528 -0.19306177  0.07574868 -0.36211299  0.4420153
  -0.36771578  0.00774803 -0.16994737 -0.15455595  0.34735941 -0.3783485 ]
 [-0.19617353 -0.10904088  0.25381021  0.09344454  0.21738228 -0.08852265
  -0.34608312  0.35290164 -0.00926217 -0.18297129  0.47061958  0.03693675
   0.05384741 -0.39880512  0.26027241 -0.10650103  0.26663327  0.09239138]
 [-0.24881555 -0.08291509 -0.33511187 -0.04496296  0.17750026 -0.24343454
   0.30587334 -0.10757747 -0.04830972 -0.09143777  0.01443588 -0.57713443
   0.04012444  0.12776929  

In [4]:
##calculating overall average face vector
transposeweightmatrix=np.transpose(weightmatrix)
avgface=np.ones((14,1))
for i in range(0,14):
    avgface[i]=np.mean(transposeweightmatrix[i])
print("The average face vector is",avgface)

##calculating average face vector for each class
classavgface=np.ones((14,9))
for j in range(0,9):
    for i in range(0,14):
        classavgface[i][j]=(int)((weightmatrix[i,2*j]/2+weightmatrix[i,2*j+1]/2))
transclassavg=np.transpose(classavgface)
#print("sdjfkl",transclassavg.shape)
storedtransclassavg=transclassavg

##subtracting average class face vector from each of the class vector
j=0
for i in range(0,n):
    transposeweightmatrix[i]=transposeweightmatrix[i]-transclassavg[j]
    if(i%2==0 and i!=0):
        j+=1


##subtracting total average face vector from class average face vector
for i in range(0,9):
    transclassavg[i]=transclassavg[i]-np.transpose(avgface)

##calculating between class scatter matrix
print(transclassavg[4].shape)
sbetweenclass=np.zeros((14,14))
for i in range(0,9):
    sbetweenclass+=np.transpose(transclassavg[i])*transclassavg[i]


##calculating within class scatter matrix
s1=np.transpose(transposeweightmatrix[0])*transposeweightmatrix[0]+np.transpose(transposeweightmatrix[1])*transposeweightmatrix[1]
s2=np.transpose(transposeweightmatrix[2])*transposeweightmatrix[2]+np.transpose(transposeweightmatrix[3])*transposeweightmatrix[3]
s3=np.transpose(transposeweightmatrix[4])*transposeweightmatrix[4]+np.transpose(transposeweightmatrix[5])*transposeweightmatrix[5]
s4=np.transpose(transposeweightmatrix[6])*transposeweightmatrix[6]+np.transpose(transposeweightmatrix[7])*transposeweightmatrix[7]
s5=np.transpose(transposeweightmatrix[8])*transposeweightmatrix[8]+np.transpose(transposeweightmatrix[9])*transposeweightmatrix[9]
s6=np.transpose(transposeweightmatrix[10])*transposeweightmatrix[10]+np.transpose(transposeweightmatrix[11])*transposeweightmatrix[11]
s7=np.transpose(transposeweightmatrix[12])*transposeweightmatrix[12]+np.transpose(transposeweightmatrix[13])*transposeweightmatrix[13]
s8=np.transpose(transposeweightmatrix[14])*transposeweightmatrix[14]+np.transpose(transposeweightmatrix[15])*transposeweightmatrix[15]
s9=np.transpose(transposeweightmatrix[16])*transposeweightmatrix[16]+np.transpose(transposeweightmatrix[17])*transposeweightmatrix[17]
swithinclass=s1+s2+s3+s4+s5+s6+s7+s8+s9
print(swithinclass.shape)
print(sbetweenclass.shape)


The average face vector is [[-74411666.74247158]
 [-56118495.35826475]
 [-49184594.20700543]
 [-49686223.45762571]
 [-46462201.81668396]
 [-49845490.00063545]
 [-56973258.43724734]
 [-56839482.02421945]
 [-54760051.27127892]
 [-45464429.12329532]
 [-51326260.59634455]
 [-53195486.41949917]
 [-59339678.83176638]
 [-59505956.63666056]]
(14,)
(14, 14)
(14, 14)


In [5]:
##calculating eigenvectors of given images
temp=np.linalg.inv(swithinclass)*sbetweenclass
ladeigenvectors,D=np.linalg.eig(temp)
print("ldaeigenvectors.shape=",ldaeigenvectors.shape)

##projecting faces onto lda-space
projectedldaeigenvectors=ldaeigenvectors*weightmatrix
print("projectedldaeigenvectors.shape=",projectedldaeigenvectors.shape)

###Testing unknown face
image=imageio.imread('Test/andrew3.png')
m,n=image.shape
image=np.reshape(image,(m*n,1))
normalisedface=image-storedavgface
print(normalisedface.shape)

#temp=np.transpose(mappedeigenvectors)*mappedeigenvectors
#tempinv=np.linalg.inv(temp)
testweight=np.transpose(mappedeigenvectors)*normalisedface
print(testweight.shape)
ldatestweight=np.transpose(projectedldaeigenvectors)*testweight

transposeweightmatrix=np.transpose(weightmatrix)
m,n=transposeweightmatrix.shape


flag=0
store=[]
for i in range(0,m):
    difference=transposeweightmatrix[i]-np.transpose(testweight)
    totaldiff=np.sum(np.power(difference,2))
    totaldiff=np.sqrt(totaldiff)
    print(totaldiff)
    store.append(totaldiff)
a=min(store)

    

NameError: name 'ldaeigenvectors' is not defined